In [1]:
import platform
import sys
import pandas as pd
import calendar
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import functions.preprocessing as prep

import sklearn
from sklearn import svm
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as sm
from sklearn import datasets
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print("This jupyter notebook is running on Python " + platform.python_version())
# It's good practice to assert packages requirements at the beginning of a script:
assert sys.version_info >= (3, 6)  # Tested with Python==3.7.9

This jupyter notebook is running on Python 3.7.9


In [29]:
training_set_file_path = 'data/training_set_VU_DM.csv'
test_set_file_path = 'data/test_set_VU_DM.csv'
training_data = pd.read_csv(training_set_file_path)
test_data = pd.read_csv(test_set_file_path)

In [30]:
training_data = prep.parseDateTime(training_data)
training_data, useless_columns = prep.removeUselessColumns(training_data, threshold=90)
training_data = prep.replaceNaN(training_data)
training_data = prep.mergeComps(training_data)
#training_data.info(verbose=True)

c:\Users\Ramon\Desktop\DMT_Asg_2\functions\preprocessing.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  return dataFrame.drop('date_time', 1)
c:\Users\Ramon\Desktop\DMT_Asg_2\functions\preprocessing.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dataFrame = dataFrame.drop(useless_columns, 1)
c:\Users\Ramon\Desktop\DMT_Asg_2\functions\preprocessing.py:20: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dataFrame = dataFrame.drop(['comp3_rate','comp3_inv'], 1)
c:\Users\Ramon\Desktop\DMT_Asg_2\functions\preprocessing.py:33: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  'comp2_rate_percent_diff', 'comp5_rate_percent_diff', 'comp8_rate_perc

Here we filter some of our data. Removing useless columns etc.

In [31]:
columns_to_drop = ['price_usd', 'month', 'year', 'srch_adults_count', 'srch_children_count', 'prop_id', 
                   'srch_room_count', 'prop_location_score2', 'prop_review_score', 'orig_destination_distance']
print(useless_columns)
temp = useless_columns[:-1]
print(temp)
#training_data = training_data.drop(columns_to_drop, 1)
#training_data.info(verbose=True)


['visitor_hist_starrating', 'visitor_hist_adr_usd', 'srch_query_affinity_score', 'comp1_rate', 'comp1_inv', 'comp1_rate_percent_diff', 'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv', 'comp4_rate_percent_diff', 'comp6_rate', 'comp6_inv', 'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv', 'comp7_rate_percent_diff', 'gross_bookings_usd']
['visitor_hist_starrating', 'visitor_hist_adr_usd', 'srch_query_affinity_score', 'comp1_rate', 'comp1_inv', 'comp1_rate_percent_diff', 'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv', 'comp4_rate_percent_diff', 'comp6_rate', 'comp6_inv', 'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv', 'comp7_rate_percent_diff']


Here we create a new column called "relevancy". A hotel can get a relevancy of 0, 1 or 5 based on whether it was booked, clicked or ignored.

In [32]:
conditions = [
    (training_data['booking_bool'] == 1),
    (training_data['click_bool'] == 1 ) & (training_data['booking_bool'] == 0),
    (training_data['click_bool'] == 0),
    ]
values = ['5', '1', '0']
training_data['relevancy'] = np.select(conditions, values)
training_data['relevancy'] = training_data['relevancy'].astype('category')

In [33]:
#training_data_y = training_data.booking_bool
#training_data_x = training_data.drop(['booking_bool'],1)
training_data_y = training_data.relevancy
#training_data_y = training_data_y[:500000]
training_data_x = training_data.drop(['click_bool', 'position', 'booking_bool', 'relevancy'],1)
training_data_x['srch_id'] = training_data_x['srch_id'].astype('category')
search_id = training_data_x.srch_id.tolist()
training_data_x = training_data_x.drop(['srch_id'],1)
training_data_x = training_data_x.drop(columns_to_drop,1)
#training_data_x = training_data_x[:500000]

training_data_x.info(verbose=True)
scaler = StandardScaler().fit(training_data_x)
training_data_x = scaler.transform(training_data_x)

search_id_test = test_data.srch_id.tolist()
prop_id_test = test_data.prop_id.tolist()
#prop_id_test = prop_id_test[:500000]
#search_id_test = search_id_test[:500000]
test_data = prep.parseDateTime(test_data)
test_data = test_data.drop(useless_columns[:-1],1)
test_data = prep.replaceNaN(test_data)
test_data = prep.mergeComps(test_data)
test_data = test_data.drop(columns_to_drop,1)
test_data = test_data.drop(['comp3_rate','comp3_inv'], 1)
test_data = test_data.drop(['srch_id'],1)
#test_data = test_data[:500000]


#test_data.info(verbose=True)

#training_data_x, testing_data_x, training_data_y, testing_data_y = train_test_split(training_data_x, training_data_y, test_size=0.10, random_state=42, stratify=training_data_y)
del training_data
#columns_to_drop = ['price_usd', 'month', 'year', 'srch_adults_count', 'srch_children_count', 'prop_id', 'srch_room_count']


c:\Users\Ramon\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """
c:\Users\Ramon\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  
c:\Users\Ramon\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  if __name__ == '__main__':


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4958347 entries, 0 to 4958346
Data columns (total 16 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   site_id                      int64  
 1   visitor_location_country_id  int64  
 2   prop_country_id              int64  
 3   prop_starrating              int64  
 4   prop_brand_bool              int64  
 5   prop_location_score1         float64
 6   prop_log_historical_price    float64
 7   promotion_flag               int64  
 8   srch_destination_id          int64  
 9   srch_length_of_stay          int64  
 10  srch_booking_window          int64  
 11  srch_saturday_night_bool     int64  
 12  random_bool                  int64  
 13  comp_rate                    float64
 14  comp_inv                     float64
 15  comp_rate_percent_diff       float64
dtypes: float64(5), int64(11)
memory usage: 605.3 MB


c:\Users\Ramon\Desktop\DMT_Asg_2\functions\preprocessing.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  return dataFrame.drop('date_time', 1)
c:\Users\Ramon\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
c:\Users\Ramon\Desktop\DMT_Asg_2\functions\preprocessing.py:33: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  'comp2_rate_percent_diff', 'comp5_rate_percent_diff', 'comp8_rate_percent_diff'], 1)
c:\Users\Ramon\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
c:\Users\Ramon\AppData\Loc

We then create and train the model

In [36]:
#clf = svm.LinearSVC() 
## 0, 1, 5
clf = svm.LinearSVC()
clf.fit(training_data_x, training_data_y)



c:\Users\Ramon\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC()

In [ ]:
#dec = clf.decision_function([1])

Make the predictions, and put them into a dataframe

In [37]:
predicted_y = clf.predict(test_data)
predicted_y_probability = clf._predict_proba_lr(test_data)

In [40]:
#print(np.unique(predicted_y))
print(predicted_y[1:500])


['1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1

In [41]:
highest_predict = np.amax(predicted_y_probability, axis=1)
myDataFrame = pd.DataFrame(list(zip(search_id_test, prop_id_test, predicted_y, highest_predict)), columns=['srch_id', 'prop_id', 'predicted_y', 'predicted_y_probability'])

In [43]:
print(myDataFrame[28:50])

    srch_id  prop_id predicted_y  predicted_y_probability
28        1   139162           1                 0.502334
29        3     6533           1                 0.526839
30        3    11425           1                 0.527138
31        3    22325           1                 0.526968
32        3    24915           1                 0.527330
33        3    27753           1                 0.527144
34        3    32382           1                 0.526905
35        3    43133           1                 0.527128
36        3    43362           1                 0.526937
37        3    45550           1                 0.527134
38        3    46861           1                 0.526493
39        3    47238           1                 0.527148
40        3    47377           1                 0.526607
41        3    63039           1                 0.526553
42        3    64168           1                 0.527215
43        3    71543           1                 0.527243
44        3   

In [44]:
myDataFrame = myDataFrame.sort_values(['srch_id', 'predicted_y'], ascending = [True, False])
#myDataFrame = myDataFrame[myDataFrame.predicted_y != 0]


In [45]:
print(myDataFrame['predicted_y'].value_counts)


1    4679343
0     279743
5         97
Name: predicted_y, dtype: int64
     srch_id  prop_id predicted_y  predicted_y_probability
100       10    39517           1                 0.491548
101       10    40475           1                 0.491721
102       10    43662           1                 0.491869
103       10    56573           1                 0.491180
104       10    59466           1                 0.492033
105       10    62403           1                 0.492051
106       10    78989           1                 0.491428
107       10    79561           1                 0.491213
108       10    80396           1                 0.491730
109       10    83789           1                 0.491283
110       10    84447           1                 0.491236
111       10    90052           1                 0.490580
112       10    92193           1                 0.491064
113       10    94138           1                 0.491806
114       10   101198           1           

In [76]:
outputFrame = myDataFrame.drop(['predicted_y', 'predicted_y_probability'],1)
outputFrame.to_csv('yooooo.csv', index=False)

c:\Users\Ramon\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
